In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
alt.data_transformers.disable_max_rows()
matplotlib.rcParams['pdf.fonttype'] = 42

%matplotlib inline

In [2]:
!ls

Airline Delays, 2004-2019.ipynb complaints.pdf
airline_delay_causes.csv        complaints.xlsx


In [3]:
df = pd.read_csv("airline_delay_causes.csv")

In [4]:
#Worst airlines by year 
#Worst airports by year
#Delays over time - worst seasons to travel (calendar heatmap)
#Sankey Diagram for each airport
#Security delays

In [7]:
df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2004,5,AA,American Airlines Inc.,JFK,"New York, NY: John F. Kennedy International",1296.0,277.0,66.43,4.00,...,0.23,39.98,1.0,6.0,13201.0,4795.0,409.0,5761.0,8.0,2228.0
1,2004,5,B6,JetBlue Airways,JFK,"New York, NY: John F. Kennedy International",2684.0,461.0,70.67,1.32,...,12.11,187.11,1.0,8.0,20322.0,3605.0,36.0,6773.0,449.0,9459.0
2,2004,5,CO,Continental Air Lines Inc.,JFK,"New York, NY: John F. Kennedy International",31.0,5.0,0.72,1.00,...,0.00,0.00,0.0,0.0,278.0,86.0,73.0,119.0,0.0,0.0
3,2004,5,DH,Atlantic Coast Airlines,JFK,"New York, NY: John F. Kennedy International",445.0,96.0,30.73,3.40,...,0.00,37.25,8.0,2.0,7433.0,1986.0,471.0,1728.0,0.0,3248.0
4,2004,5,DL,Delta Air Lines Inc.,JFK,"New York, NY: John F. Kennedy International",1175.0,317.0,51.60,1.93,...,0.85,35.31,3.0,2.0,13689.0,3113.0,57.0,8289.0,28.0,2202.0


In [10]:
df.dtypes

year                     int64
 month                   int64
carrier                 object
carrier_name            object
airport                 object
airport_name            object
arr_flights            float64
arr_del15              float64
carrier_ct             float64
 weather_ct            float64
nas_ct                 float64
security_ct            float64
late_aircraft_ct       float64
arr_cancelled          float64
arr_diverted           float64
 arr_delay             float64
 carrier_delay         float64
weather_delay          float64
nas_delay              float64
security_delay         float64
late_aircraft_delay    float64
dtype: object

In [22]:
df=df.rename(columns={' month': 'month',
                     ' weather_ct': 'weather_ct',
                     ' arr_delay': 'arr_delay',
                     ' carrier_delay': 'carrier_delay'})

In [26]:
df.drop(['airport_name'], axis=1)

,year,month,carrier,carrier_name,airport,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2004,5,AA,American Airlines Inc.,JFK,1296.0,277.0,66.43,4.00,166.35,0.23,39.98,1.0,6.0,13201.0,4795.0,409.0,5761.0,8.0,2228.0
1,2004,5,B6,JetBlue Airways,JFK,2684.0,461.0,70.67,1.32,189.79,12.11,187.11,1.0,8.0,20322.0,3605.0,36.0,6773.0,449.0,9459.0
2,2004,5,CO,Continental Air Lines Inc.,JFK,31.0,5.0,0.72,1.00,3.28,0.00,0.00,0.0,0.0,278.0,86.0,73.0,119.0,0.0,0.0
3,2004,5,DH,Atlantic Coast Airlines,JFK,445.0,96.0,30.73,3.40,24.62,0.00,37.25,8.0,2.0,7433.0,1986.0,471.0,1728.0,0.0,3248.0
4,2004,5,DL,Delta Air Lines Inc.,JFK,1175.0,317.0,51.60,1.93,227.31,0.85,35.31,3.0,2.0,13689.0,3113.0,57.0,8289.0,28.0,2202.0
5,2004,5,EV,Atlantic Southeast Airlines,JFK,150.0,50.0,14.12,9.03,22.47,0.00,4.39,4.0,2.0,2075.0,574.0,664.0,709.0,0.0,128.0
6,2004,5,HP,America West Airlines Inc.,JFK,341.0,130.0,27.40,0.11,88.40,0.00,14.09,5.0,2.0,6390.0,2092.0,10.0,3560.0,0.0,728.0
7,2004,5,MQ,American Eagle Airlines Inc.,JFK,394.0,94.0,19.10,3.09,39.72,0.00,32.09,7.0,2.0,4966.0,787.0,289.0,1817.0,0.0,2073.0
8,2004,5,NW,Northwest Airlines Inc.,JFK,93.0,26.0,12.61,1.68,9.88,0.40,1.43,4.0,1.0,1293.0,575.0,155.0,522.0,16.0,25.0
9,2004,5,OH,Comair Inc.,JFK,250.0,89.0,23.13,29.24,32.89,0.85,2.90,6.0,0.0,5615.0,1167.0,2360.0,1915.0,50.0,123.0


In [21]:
df.groupby(by='airport')[' weather_ct'].sum()

airport
EWR    16073.30
JFK    12486.69
LGA    18978.20
Name:  weather_ct, dtype: float64